In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd gdrive/My Drive/code/
    %cd photovoltaic_images
    root_folder = "/data"
else:
    print('Not running on CoLab')
    root_folder = "data"

Not running on CoLab


In [2]:
#!pip install git+https://github.com/tensorflow/examples.git

In [3]:
#!pip install pydot
#!pip install pydotplus
#!pip install graphviz

In [4]:
#!pip install imantics

In [5]:
from pycocotools.coco import COCO
import numpy as np

import random
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pydot
from imantics import Polygons, Mask
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
import cv2

## For visualizing results

import matplotlib.gridspec as gridspec
import sys
%matplotlib inline

In [6]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd gdrive/My Drive/code/photovoltaic_images
else:
  print('Not running on CoLab')

Not running on CoLab


In [7]:
random.seed(123)

In [8]:
from utils import *
from models.model import *
import rectification as rec

In [9]:
# Main Parameter
# Segmentation
model_name = 'Mobile_Net_v2'
EPOCHS = 1000

# Regression
nr_images_batch = 1000
nr_batch = 45
nr_epoch = 10

In [10]:
model = keras.models.load_model('models/' + str(model_name) + '_' + str(EPOCHS))

In [11]:
# Parameter
x = 192
y = 192

batch_size = 1
input_image_size = (x,y)
OUTPUT_CHANNELS = 1

folder = 'data/Rathausen_192x192'
mode_train = 'train'

mode_val = 'val'
mode_test = 'test'

folder_deepsolareye = 'data/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages'

STEPS_PER_EPOCH = 250
#VAL_SUBSPLITS = 5
VALIDATION_STEPS = 4

In [12]:
#model = eval(model_name + '([192,192,3])')

In [13]:
log = RuntimeLogger(".", model_name, EPOCHS)
log.start()

In [14]:
images_train, dataset_size_train, coco = filterDataset(folder,  mode_train)
#coco_train = copy.deepcopy(coco)
coco_train = coco

images_val, dataset_size_val, coco = filterDataset(folder,  mode_val)
#coco_val = copy.deepcopy(coco)
coco_val = coco

images_test, dataset_size_test, coco = filterDataset(folder,  mode_test)
#coco_val = copy.deepcopy(coco)
coco_test = coco

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [15]:
images_deepsolareye, dataset_size_deepsolareye, coco = filterDatasetDeepSolarEye(folder_deepsolareye)
#coco_val = copy.deepcopy(coco)
coco_deepsolar = coco

In [16]:
gen_train = dataGeneratorCoco(images_train, coco_train, folder,
                            input_image_size, batch_size, mode_train)
#gen1 = dataGeneratorCoco(images, classes, coco, folder,
#                            input_image_size, 1, 'test', mask_type)
gen_val = dataGeneratorCoco(images_val, coco_val, folder,
                            input_image_size, batch_size, mode_val)
gen_test = dataGeneratorCoco(images_test, coco_test, folder,
                            input_image_size, batch_size, mode_test)

In [17]:
def show_predictions(model, dataset=None, num=4):
    """[summary]

    Args:
        dataset ([type], optional): [description]. Defaults to None.
        num (int, optional): [description]. Defaults to 4.
    """
    if dataset:
        for i in range(0, num):
            image, mask = next(dataset)
            pred_mask = model.predict(image)
            display([image[0], mask[0], pred_mask[0]])
    else:
        img = model.predict(sample_image)
        display([sample_image[0], sample_mask[0],
             img[0]])

## Evaluate Bildsegmentierung

# Store, Registration and Recification

In [18]:
predict_and_store(model, model_name + str(EPOCHS), 'Rathausen_192x192', images_test, dataset_size_test, folder, x, y, root_folder)

NameError: name 'model' is not defined

In [ ]:
training_start_time = time()
predict_and_store(model, model_name + str(EPOCHS), 'DeepSolarEye_192x192', images_deepsolareye, dataset_size_deepsolareye, folder_deepsolareye, x, y, root_folder)
training_time = time() - training_start_time
log.log_task_end('predict_deepsolareye_time', training_start_time)
print("--- %s seconds ---" % (training_time))

# Regression

In [14]:
# Make Regression on recification deep solar eye pictures
folder = "data/" + model_name + str(EPOCHS) + "/DeepSolarEye_192x192_rec"
#folder = "data/DeepSolarEyeCalc/DeepSolarEye_192x192_rec"

In [15]:
folder

'data/Mobile_Net_v21000/DeepSolarEye_192x192_rec'

In [16]:
training_start_time = time()
metrics = train_keras(model_name, folder, nr_images_batch, nr_batch, nr_epoch, 10)
training_time = time() - training_start_time
log.log_task_end('regression_time', training_start_time)
print("--- %s seconds ---" % (training_time))

--- Batch 0 --- 
select from to 0 1000
Files in train_files: 1000
25 images to array
50 images to array
75 images to array
100 images to array
125 images to array
150 images to array
175 images to array
200 images to array
225 images to array
250 images to array
275 images to array
300 images to array
325 images to array
350 images to array
375 images to array
400 images to array
425 images to array
450 images to array
475 images to array
500 images to array
525 images to array
550 images to array
575 images to array
600 images to array
625 images to array
650 images to array
675 images to array
700 images to array
725 images to array
750 images to array
775 images to array
800 images to array
825 images to array
850 images to array
875 images to array
900 images to array
925 images to array
950 images to array
975 images to array
1000 images to array
All images to array!

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/n...|False       

KeyboardInterrupt: 

## Evaluate Regression

In [ ]:
# Print Model Summary:
print("MODEL SUMMARY")
print("-------------")
print("Test MSE:", metrics[1])
print("Test RMSE:", metrics[2])
print("Test MAE:", metrics[3])


In [ ]:
# add Metric to the log
log.add_metric('Regression MSE', metrics[1])
log.add_metric('Regression RMSE', metrics[2])
log.add_metric('Regression MAE', metrics[3])

# End of Experiment

In [ ]:
log.log_experiment_end()